# EOPF S3 OLCI L1 Product Data Structure Proposal


In [1]:
import os
import xarray as xr
import glob
from utils import get_ds_variables as get_ds
from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder

In [2]:
path_to_product = glob.glob("data/S3A_SL_1_RBT*.SEN3")[0]

# Groups definition
groups = {}
for g in ['an','ao','bn','bo','in','io','fn','fo']:
    groups['coordinates_%s' % g] = get_ds(path_to_product, names=["geodetic_%s" % g, "cartesian_%s" % g])
groups['tiepoints'] = get_ds(path_to_product, names=["geodetic_tx", "cartesian_tx"])

for g in ['an','ao']:
    groups['measurements_%s' % g] = get_ds(path_to_product, names=["%s_radiance_%s" % (r,g) for r in ["S1","S2","S3","S4","S5","S6"]]+["flags_%s" % g])
for g in ['bn','bo']:
    groups['measurements_%s' % g] = get_ds(path_to_product, names=["%s_radiance_%s" % (r,g) for r in ["S4","S5","S6"]]+["flags_%s" % g])
for g in ['in','io']:
    groups['measurements_%s' % g] = get_ds(path_to_product, names=["%s_BT_%s" % (r,g) for r in ["S7","S8","S9","F2"]]+["flags_%s" % g])
for g in ['fn','fo']:
    groups['measurements_%s' % g] = get_ds(path_to_product, names=["%s_BT_%s" % (r,g) for r in ["F1"]]+["flags_%s" % g])

groups['meteo'] = get_ds(path_to_product, names=0["met_tx"])
for g in ['tn','to']:
    groups['geometry_%s' % g] = get_ds(path_to_product, names=["geometry_%s" % g])
for g in ['an','bn','in']:
    groups['time_%s' % g] = get_ds(path_to_product, names=["time_%s" % g])

for v in ["S1","S2","S3","S4","S5","S6","S7","S8","S9","F2","F1"]:
    groups["%s_quality" % v] = get_ds(path_to_product, names=["%s_quality_%s" % (v,r) for r in ['an','ao','bn','bo','in','io','fn','fo']])
for g in ['an','ao','bn','bo','in','io','fn','fo']:
    groups['instrument_%s' % g] = get_ds(path_to_product, names=["indices_%s" % g])
for g in ['an','ao']:
    groups['orphan_%s' % g] = get_ds(path_to_product, names=["%s_radiance_%s" % (r,g) for r in ["S1","S2","S3","S4","S5","S6"]]+["%s_quality_%s" % (r,g) for r in ["S1","S2","S3","S4","S5","S6"]]+["flags_%s" % g], orphan=True)
for g in ['bn','bo']:
    groups['orphan_%s' % g] = get_ds(path_to_product, names=["%s_radiance_%s" % (r,g) for r in ["S4","S5","S6"]]+["%s_quality_%s" % (r,g) for r in ["S4","S5","S6"]]+["flags_%s" % g], orphan=True)
for g in ['in','io']:
    groups['orphan_%s' % g] = get_ds(path_to_product, names=["%s_BT_%s" % (r,g) for r in ["S7","S8","S9","F2"]]+["%s_quality_%s" % (r,g) for r in ["S7","S8","S9","F2"]]+["flags_%s" % g], orphan=True)
for g in ['fn','fo']:
    groups['orphan_%s' % g] = get_ds(path_to_product, names=["%s_BT_%s" % (r,g) for r in ["F1"]]+["%s_quality_%s" % (r,g) for r in ["F1"]]+["flags_%s" % g], orphan=True)

# Create a new EOProduct instance
product_name = os.path.basename("S3_SLSTR_L1")
product = EOProductBuilder("S3_SLSTR_L1", coords=EOGroupBuilder('coords'))
# do the same work as before 
product.metadatas = ["xfdumanifest.xml"]

# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/S1_radiance_an.nc",decode_times=False,mask_and_scale=False)
product.attrs = {att: f'{value[:50]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}
    
# ==================== Product groups setting ========================

for group_name, ds in groups.items():
    group = EOGroupBuilder(group_name)
    group.attrs['description'] = f'{group_name} Data Group'
    group.dims = ds.dims

    for v in ds.variables:
        variable = EOVariableBuilder(v, default_attrs = False)
        var = ds.get(v)
        variable.dtype = var.dtype
        variable.dims = var.dims
        variable.attrs = var.attrs

        group.variables.append(variable)

    product.groups.append(group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

<>:18: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:18: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?


IndexError: list index out of range

In [ ]:
product.compute()